In [1]:
import cirq
from cirq.contrib import paulistring

In [2]:
def ops_to_matrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    return c.to_unitary_matrix()
q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
q2 = cirq.NamedQubit('q2')
q3 = cirq.NamedQubit('q3')
q4 = cirq.NamedQubit('q4')
q5 = cirq.NamedQubit('q5')

def cz_count(circuit):
    assert all(isinstance(op, cirq.GateOperation) and
               isinstance(op.gate, (cirq.Rot11Gate, cirq.google.Exp11Gate))
               for op in circuit.all_operations()
               if len(op.qubits) >= 2), 'Circuit not decomposed to CZs'
    return sum(len(op.qubits) == 2 for op in circuit.all_operations())

def stats(circuit):
    if not all(isinstance(op, cirq.GateOperation) and
               isinstance(op.gate, (cirq.Rot11Gate, cirq.google.Exp11Gate))
               for op in circuit.all_operations()
               if len(op.qubits) >= 2):
        circuit = cirq.Circuit(circuit)
        try:
            cirq.ExpandComposite().optimize_circuit(circuit)
        except:
            print('Failed to expand circuit')
        
        try:
            cirq.google.ConvertToXmonGates().optimize_circuit(circuit)
        except:
            print('Failed to convert to Xmon')
        
        try:
            circuit = cirq.google.optimized_for_xmon(circuit)
        except:
            print('Failed to optimize for Xmon')
    print('  CZ count:', cz_count(circuit))
    print('  Depth:', len(circuit))
    
def optimize(circuit):
    print('Before')
    stats(circuit)
    c_opt = paulistring.optimized_circuit(circuit)
    print('After')
    stats(c_opt)
    display(circuit)
    display(c_opt)
    return c_opt

In [3]:
import openfermion

# Set parameters of jellium model.
wigner_seitz_radius = 5. # Radius per electron in Bohr radii.
n_dimensions = 2 # Number of spatial dimensions.
grid_length = 2 # Number of grid points in each dimension.
spinless = True # Whether to include spin degree of freedom or not.
n_electrons = 2 # Number of electrons.

# Figure out length scale based on Wigner-Seitz radius and construct a basis grid.
length_scale = openfermion.wigner_seitz_length_scale(
    wigner_seitz_radius, n_electrons, n_dimensions)
grid = openfermion.Grid(n_dimensions, grid_length, length_scale)

# Initialize the model and print out.
fermion_hamiltonian = openfermion.jellium_model(grid, spinless=spinless, plane_wave=False)
print(fermion_hamiltonian)

# Convert to DiagonalCoulombHamiltonian type.
hamiltonian = openfermion.get_diagonal_coulomb_hamiltonian(fermion_hamiltonian)

0.1256637061435917 [0^ 0] +
-0.07957747154594769 [0^ 0 1^ 1] +
-0.07957747154594769 [0^ 0 2^ 2] +
-0.23873241463784306 [0^ 0 3^ 3] +
-0.06283185307179587 [0^ 1] +
-0.06283185307179585 [0^ 2] +
-0.06283185307179587 [1^ 0] +
0.1256637061435917 [1^ 1] +
-0.07957747154594769 [1^ 1 0^ 0] +
-0.23873241463784306 [1^ 1 2^ 2] +
-0.07957747154594769 [1^ 1 3^ 3] +
-0.06283185307179585 [1^ 3] +
-0.06283185307179585 [2^ 0] +
0.1256637061435917 [2^ 2] +
-0.07957747154594769 [2^ 2 0^ 0] +
-0.23873241463784306 [2^ 2 1^ 1] +
-0.07957747154594769 [2^ 2 3^ 3] +
-0.06283185307179587 [2^ 3] +
-0.06283185307179585 [3^ 1] +
-0.06283185307179587 [3^ 2] +
0.1256637061435917 [3^ 3] +
-0.23873241463784306 [3^ 3 0^ 0] +
-0.07957747154594769 [3^ 3 1^ 1] +
-0.07957747154594769 [3^ 3 2^ 2]


In [4]:
import cirq
import openfermioncirq

# Obtain the Bogoliubov transformation matrix.
quadratic_hamiltonian = openfermion.QuadraticHamiltonian(hamiltonian.one_body)
transformation_matrix = quadratic_hamiltonian.diagonalizing_bogoliubov_transform()

# Create a circuit that prepares the mean-field state
occupied_orbitals = range(n_electrons)
n_qubits = openfermion.count_qubits(quadratic_hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)
state_preparation_circuit = cirq.Circuit.from_ops(
    openfermioncirq.bogoliubov_transform(
        qubits, transformation_matrix, initial_state=occupied_orbitals))

# Print circuit.
cirq.DropNegligible().optimize_circuit(state_preparation_circuit)
print(state_preparation_circuit)

0: ──────────────YXXY─────────────────────────────────
                 │
1: ───YXXY───────#2^-0.502───────────────YXXY─────────
      │                                  │
2: ───#2^0.995───────────────YXXY────────#2^0.00482───
                             │
3: ──────────────────────────#2^-0.498────────────────


In [5]:
c_opt = optimize(state_preparation_circuit)

Before
Failed to expand circuit
  CZ count: 8
  Depth: 14
After
  CZ count: 8
  Depth: 12


0: ──────────────YXXY─────────────────────────────────
                 │
1: ───YXXY───────#2^-0.502───────────────YXXY─────────
      │                                  │
2: ───#2^0.995───────────────YXXY────────#2^0.00482───
                             │
3: ──────────────────────────#2^-0.498────────────────

0: ───(Z^-0.5-X^-0.5)────────────────────────@───[X]^0.251────@───(X^0.5-Z^0.5)──────────────────────────────────────────
                                             │                │
1: ───(Z^-0.5-X^0.5)────@───[X]^-0.498───@───@───[X]^0.251────@───Y^-0.5──────────@───[X]^0.00241────@───X^-0.5──────────
                        │                │                                        │                  │
2: ───Y^0.5─────────────@───[Y]^0.498────@───@───[Y]^-0.249───@───X^-0.5──────────@───[Y]^-0.00241───@───(Z^0.5-X^0.5)───
                                             │                │
3: ───Y^-0.5─────────────────────────────────@───[Y]^-0.249───@───Y^0.5──────────────────────────────────────────────────

In [6]:
from openfermioncirq import trotter

# Set algorithm parameters.
time = 1.0
n_steps = 1
order = 1

# Construct circuit
swap_network_trotter_step = cirq.Circuit.from_ops(
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order,
        algorithm=trotter.LINEAR_SWAP_NETWORK),
    strategy=cirq.InsertStrategy.EARLIEST)

# Print circuit.
cirq.DropNegligible().optimize_circuit(swap_network_trotter_step)
print(swap_network_trotter_step.to_text_diagram(transpose=True))

0       1          2       3
│       │          │       │
XXYY────XXYY^-0.02 XXYY────XXYY^-0.02
│       │          │       │
@───────@^0.0253   @───────@^0.0253
│       │          │       │
×ᶠ──────×ᶠ         ×ᶠ──────×ᶠ
│       │          │       │
│       @──────────@^0.076 │
│       │          │       │
│       ×ᶠ─────────×ᶠ      │
│       │          │       │
XXYY────XXYY^-0.02 XXYY────XXYY^-0.02
│       │          │       │
@───────@^0.0253   @───────@^0.0253
│       │          │       │
×ᶠ──────×ᶠ         ×ᶠ──────×ᶠ
│       │          │       │
Z^-0.04 │          │       Z^-0.04
│       │          │       │
│       @──────────@^0.076 │
│       │          │       │
│       ×ᶠ─────────×ᶠ      │
│       │          │       │
│       Z^-0.04    Z^-0.04 │
│       │          │       │
│       @──────────@^0.076 │
│       │          │       │
│       ×ᶠ─────────×ᶠ      │
│       │          │       │
@───────@^0.0253   @───────@^0.0253
│       │          │       │
XXYY────XXYY^-0.02 XXYY──

In [7]:
c_opt = optimize(swap_network_trotter_step)

Before
Failed to expand circuit
  CZ count: 32
  Depth: 42
After
  CZ count: 32
  Depth: 51


0: ───XXYY─────────@──────────×ᶠ──────────────────XXYY─────────@──────────×ᶠ───Z^-0.04───────────────────────────────────────────@──────────XXYY─────────×ᶠ──────────────────@──────────XXYY─────────×ᶠ───
      │            │          │                   │            │          │                                                      │          │            │                   │          │            │
1: ───XXYY^-0.02───@^0.0253───×ᶠ───@─────────×ᶠ───XXYY^-0.02───@^0.0253───×ᶠ─────────────@─────────×ᶠ───Z^-0.04───@─────────×ᶠ───@^0.0253───XXYY^-0.02───×ᶠ───@─────────×ᶠ───@^0.0253───XXYY^-0.02───×ᶠ───
                                   │         │                                           │         │              │         │                                 │         │
2: ───XXYY─────────@──────────×ᶠ───@^0.076───×ᶠ───XXYY─────────@──────────×ᶠ─────────────@^0.076───×ᶠ───Z^-0.04───@^0.076───×ᶠ───@──────────XXYY─────────×ᶠ───@^0.076───×ᶠ───@──────────XXYY─────────×ᶠ───
      │            │          │                   │            │          │                                                      │          │            │                   │          │            │
3: ───XXYY^-0.02───@^0.0253───×ᶠ──────────────────XXYY^-0.02───@^0.0253───×ᶠ───Z^-0.04───────────────────────────────────────────@^0.0253───XXYY^-0.02───×ᶠ──────────────────@^0.0253───XXYY^-0.02───×ᶠ───

0: ───[Z]^-0.257───X^0.5───@───(Z-Y^-0.5)───@───[Z]^-0.49────X^0.5───@───[Z]^-0.0196───X^0.5──────────────────────────────────────────────────────────────@───────────(Z-Y^-0.5)───@───────────[Z]^-0.49───X^0.5───@───────────[Z]^-0.534───X^0.5─────────────────────────@───────[X]^0.51───@──────────[X]^0.0253────────────────@───────[X]^0.513────────────────────────────────────────────────────────────────────────────@───────[Y]^0.51─────@──────────[Y]^-0.987────────────────@───────[Y]^-0.73───X^0.5────────────────
                           │                │                        │                                                                                    │                        │                               │                                                      │                  │                                    │                                                                                            │                    │                                    │
1: ───[Z]^0.243────X^0.5───@───[X]^0.51─────@───[X]^-0.987───────────@───Y^-0.5────────@───────Y^-0.5───@───Y^0.5────────────────@───[Z]^-0.442───X^0.5───@───────────[X]^-0.51────@───────────────────────────────@───────────@────────────[X]^-0.076───@───[X]^-0.958───@───────X^-0.5─────@──────────[Z]^0.01─────X^0.5────────@───────X^0.5─────────────────────@───────X^0.5────@───[Z]^-0.538───X^0.5───@───[Y]^-0.551───@───────(Z-Y^-0.5)───@──────────[Z]^-0.49────X^0.5────────@───────[Z]^0.73────X────────────────────
                                                                                       │                │                        │                                                                                             │                         │                                                                                                          │                │                        │
2: ───[Z]^-0.257───X^0.5───@───(Z-Y^-0.5)───@───[Z]^-0.49────X^0.5───@───X^0.5─────────@───────X^0.5────@───[Z]^-0.538───X^0.5───@───[Y]^-0.558───@───────Y^-0.5──────@────────────[Z]^-0.49───X^0.5───────@───────────────────@─────────────────────────@───[Z]^-0.458───X^0.5───@──────────[X]^0.51───@────────────[X]^0.0253───────────@───────────Y^-0.5────────@───────Y^-0.5───@───Y^0.5────────────────@───[Z]^0.551────X^0.5───@────────────[X]^0.51───@────────────[X]^-0.987───────────@───────────[X]^-0.23───Y^-0.5───
                           │                │                        │                                                                            │                   │                                    │                                                                      │                     │                                 │                                                                                            │                       │                                 │
3: ───[Z]^0.243────X^0.5───@───[X]^0.51─────@───[X]^-0.987───────────@───[X]^-0.48────────────────────────────────────────────────────────────────@───────[Y]^-0.51───@────────────────────────────────────@───────[Y]^0.966──────────────────────────────────────────────────────@──────────Y^-0.5─────@────────────[Z]^-0.49────X^0.5───@───────────[Z]^-0.0127───X^0.5──────────────────────────────────────────────────────────────@────────────Y^-0.5─────@────────────[Z]^-0.49────X^0.5───@───────────[Z]^0.73────X────────

In [8]:
split_operator_trotter_step = cirq.Circuit.from_ops(
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order,
        algorithm=trotter.SPLIT_OPERATOR),
    strategy=cirq.InsertStrategy.EARLIEST)
cirq.DropNegligible().optimize_circuit(split_operator_trotter_step)
print(split_operator_trotter_step.to_text_diagram(transpose=True))

0       1             2           3
│       │             │           │
│       │             YXXY────────#2^0.5
│       │             │           │
│       YXXY──────────#2^0.608    │
│       │             │           │
│       │             YXXY────────#2^-0.00416
│       │             │           │
YXXY────#2^0.667      │           │
│       │             │           │
Z       YXXY──────────#2^0.612    │
│       │             │           │
│       │             YXXY────────#2^-0.498
│       │             │           │
Z       Z^-0.02       │           │
│       │             │           │
│       │             Z^-0.02     Z^-0.04
│       │             │           │
│       │             YXXY────────#2^0.498
│       │             │           │
│       YXXY──────────#2^-0.612   │
│       │             │           │
YXXY────#2^-0.667     │           │
│       │             │           │
│       │             YXXY────────#2^0.00416
│       │             │           │
│       YXXY───────

In [9]:
c_opt = optimize(split_operator_trotter_step)

Before
Failed to expand circuit
Failed to optimize for Xmon
  CZ count: 90
  Depth: 339
After
  CZ count: 76
  Depth: 106


0: ─────────────────────────────────────YXXY───────Z──────────────────────Z──────────────────────────────────────────YXXY─────────────────────────────────@──────────×──────────────────────────────────────@──────────×─────────────────#2──────────────────────#2────────────────────────────────────────#2──────────────────────Z^-0.04───#2───────────────────────────────────────#2───────────────────────────────#2──────────×───────×───────
                                        │                                                                            │                                    │          │                                      │          │                 │                       │                                         │                                 │                                        │                                │           │       │
1: ────────────YXXY─────────────────────#2^0.667───YXXY───────────────────Z^-0.02────────────────────────YXXY────────#2^-0.667────────────────YXXY────────@^0.0507───×────────────────────────@─────────×───@^0.0507───×───@─────────×───YXXY^0.5───#2───────────YXXY^-0.00416────────────────#2───────────YXXY^-0.498─────────────Z^-0.02───YXXY^0.498───#2──────────────────────────YXXY^0.00416───#2────────────────YXXY^-0.5───×───×───×───×───
               │                                   │                                                     │                                    │                                               │         │                  │         │              │                                         │                                                           │                                          │                                 │       │
2: ───YXXY─────#2^0.608───YXXY─────────────────────#2^0.612───YXXY──────────────────Z^-0.02───YXXY───────#2^-0.612───────────────YXXY─────────#2^-0.608──────────────YXXY──────@──────────×───@^0.152───×───@──────────×───@^0.152───×──────────────YXXY^0.608───────────────────#2───────────YXXY^0.612─────────────────Z^-0.02──────────────────────────YXXY^-0.612───#2───────────────────────────YXXY^-0.608───×───────────────────×───×───×───
      │                   │                                   │                               │                                  │                                   │         │          │                 │          │                                                         │                                                                                      │                                          │                       │
3: ───#2^0.5──────────────#2^-0.00416─────────────────────────#2^-0.498─────────────Z^-0.04───#2^0.498───────────────────────────#2^0.00416──────────────────────────#2^-0.5───@^0.0507───×─────────────────@^0.0507───×─────────────────────────────────────────────────────────YXXY^0.667───Z──────────────────────────Z──────────────────────────────────────────────YXXY^-0.667────────────────────────────────×───────────────────────×───────

0: ───X^0.5──────────────────────────────────────────────────────────────────────@───[X]^0.333──────@───Y^0.5───────────────────────────────────────────────────────────────────────────────────────────@───[X]^0.333──────@───X^-0.5────────────────────────────@────────────X^-0.5───@────────────X^0.5────────────────@──────────────────────────────────────────────────────────────────────@───────X^-0.5───@───────────X^-0.5───────────────────@───[Z]^0.902───X^0.5──────────────────────────────────────────────────@───[X]^0.25────@───(Z^0.5-X^0.5)─────────────────────────────@───[Y]^-0.00208───@───[X]^-0.792────────────────────────────────@───[Y]^-0.01───@───[X]^0.322─────────────────────────@───[Y]^-0.00208───@───Y^0.5──────────────────────────────────────────@───────[Y]^-0.75───@─────────────────[Y]^0.75───@────────────────────────────────@───X^-0.5───@───X^0.5─────────────@───(X-Z^0.5)──────────────────────────────────────────────
                                                                                 │                  │                                                                                                   │                  │                                     │                     │                                 │                                                                      │                │                                    │                                                                      │               │                                             │                  │                                             │               │                                     │                  │                                                  │                   │                            │                                │            │                     │
1: ───(Z^-0.5-X^0.5)───────────────────────@───────[X]^-0.304───@───(Z-Y^-0.5)───@───[X]^0.333──────@───@────────────[X]^0.306────@───[Y]^0.48─────────────────────────────────────@───[X]^0.306────@───@───[X]^0.333──────@───(Z-Y^-0.5)───@───[X]^-0.304───@───@────────────X^0.5────@────────────[Z]^0.0253───X^0.5───@────────────────────────@───────Y^-0.5───@───Y^0.5────────@───X^0.5───@───────X^0.5────@───────────[Z]^0.0253───X^0.5───────@───@───────────Y^-0.5───@───(Z^0.5-X^-0.5)───@───[Z]^0.402────X^0.5───@───[X]^-0.25───@───@───────────────[X]^-0.304───@───Y^-0.5───@───[X]^0.00208────@───@────────────[X]^-0.306───@───[Y]^0.292───@───[X]^0.01────@───[Y]^-0.822───@───[X]^-0.306───@───@───[X]^0.00208────@───Y^0.5────@───[X]^-0.304───@───(Z^-0.5-X^0.5)───@───────Y^-0.5──────@─────────────────Y^0.5──────@───@───X^-0.5───@───X^0.5───@───@───X^0.5────@───X^-0.5────────────@───@───────────X^-0.5───@───X^-0.5───@───(X^0.5-Z^-0.5)───
                                           │                    │                                       │                         │                                                │                │                                       │                │                                                                                    │                │                │                                                                     │                    │                    │                                            │                            │                                   │                         │                                                │                │                                   │                │                                                                         │            │           │                                          │                    │            │
2: ───(Z^0.5-X^-0.5)───@───[X]^-0.25───@───@───────[X]^-0.304───@───Y^-0.5───────@───[X]^0.00208────@───@────────────[X]^-0.306───@───[Y]^0.292───@───[X]^0.01────@───[Y]^-0.822───@───[X]^-0.306───@───@───[X]^0.00208────@───Y^0.5────────@───[X]^-0.304───@───[Y]^0.0283───@────────X^-0.5────────────────────@───────[Z]^-0.975───X^0.

In [10]:
order=3
n_steps=1
swap_network_trotter_step = cirq.Circuit.from_ops(
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order,
        algorithm=trotter.LINEAR_SWAP_NETWORK),
    strategy=cirq.InsertStrategy.EARLIEST)
cirq.DropNegligible().optimize_circuit(swap_network_trotter_step)
print(swap_network_trotter_step.to_text_diagram(transpose=True))

0          1             2          3
│          │             │          │
XXYY───────XXYY^-0.00309 XXYY───────XXYY^-0.00309
│          │             │          │
@──────────@^0.00392     @──────────@^0.00392
│          │             │          │
×ᶠ─────────×ᶠ            ×ᶠ─────────×ᶠ
│          │             │          │
│          @─────────────@^0.0118   │
│          │             │          │
│          ×ᶠ────────────×ᶠ         │
│          │             │          │
XXYY───────XXYY^-0.00309 XXYY───────XXYY^-0.00309
│          │             │          │
@──────────@^0.00392     @──────────@^0.00392
│          │             │          │
×ᶠ─────────×ᶠ            ×ᶠ─────────×ᶠ
│          │             │          │
Z^-0.00619 │             │          Z^-0.00619
│          │             │          │
│          @─────────────@^0.0118   │
│          │             │          │
│          ×ᶠ────────────×ᶠ         │
│          │             │          │
│          Z^-0.00619    Z^-0.00619 │

In [11]:
c_opt = optimize(swap_network_trotter_step)

Before
Failed to expand circuit
Failed to optimize for Xmon
  CZ count: 1300
  Depth: 3825
After
  CZ count: 656
  Depth: 984


0: ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00982─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY───────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00982─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY───────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00816─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY───────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00816─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY───────────×ᶠ───XXYY────────────@──────────×ᶠ───────────────────XXYY────────────@──────────×ᶠ───Z^-0.013──────────────────────────────────────────────@──────────XXYY────────────×ᶠ───────────────────@──────────XXYY────────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00816─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY───────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00816─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY───────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY────────────@───────────×ᶠ───────────────────XXYY────────────@───────────×ᶠ───Z^-0.00619────────────────────────────────────────────────@───────────XXYY────────────×ᶠ───────────────────@───────────XXYY────────────×ᶠ───XXYY───────────@────────────×ᶠ────────────────────XXYY───────────@────────────×ᶠ───Z^0.00982─────────────────────────────────────────────────@────────────XXYY───────────×ᶠ────────────────────@────────────XXYY────────

0: ───[Z]^-0.25───X^0.5───@───(Z-Y^-0.5)───@───[Z]^-0.498───X^0.5───@───[Z]^-0.00115───X^0.5───────────────────────────────────────────────────────@───────────Y^-0.5──────@────────────[Z]^-0.498───X^0.5───@──────────────[Z]^-0.497───X^0.5──────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@───────(Z-Y^-0.5)───────────────────────────────────────────────────────────@───────────[Z]^0.252────────[X]^0.997───@────────────[X]^-0.997───────────@───────────[Z]^0.753───X^0.5───────────────────────────────────────────────────────────────@───────────(Z-Y^-0.5)───@────────────[Z]^-0.498───X^0.5───@──────────────[Z]^-0.497───X^0.5────────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@───────(Z-Y^-0.5)─────────────────────────────────────────────────────────────@─────────────[Z]^0.00196─────[X]^0.999───@───────────[X]^0.999────────────@────────────[Z]^-0.501───X^0.5────────────────────────────────────────────────@────────────(Z-Y^-0.5)───@─────────────[Z]^-0.498───X^0.5───@───────────[Z]^-0.00671───X^0.5──────────────────────────@───────[X]^-0.502───@────────────[X]^0.00622─────────────────@───────(Z-Y^-0.5)───────────────────────────────────────────────────────────────────────────@────────────[Z]^-0.00311───[X]^0.999───@────────────[X]^0.999─────────────@──────────────[Z]^0.998───X^0.5───────────────────────────────────────────────────────────────@────────────(Z-Y^-0.5)───@────────────[Z]^-0.498───X^0.5───@─────────────[Z]^-0.497───X^0.5────────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@────────(Z-Y^-0.5)───────────────────────────────────────────────────────────@───────────[Z]^0.252───────[X]^0.997───@────────────[X]^-0.997───────────@────────────[Z]^0.753───X^0.5───────────────────────────────────────────────────────────────@────────────(Z-Y^-0.5)───@────────────[Z]^-0.498───X^0.5───@─────────────[Z]^-0.497───X^0.5────────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@────────(Z-Y^-0.5)───────────────────────────────────────────────────────────@───────────[Z]^0.252────────[X]^0.997───@────────────[X]^-0.997───────────@────────────[Z]^0.753───X^0.5───────────────────────────────────────────────────────────────@────────────(Z-Y^-0.5)───@────────────[Z]^-0.498───X^0.5───@─────────────[Z]^-0.497───X^0.5────────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@────────(Z-Y^-0.5)───────────────────────────────────────────────────────────@───────────[Z]^0.252────────[X]^0.997───@────────────[X]^-0.997───────────@────────────[Z]^0.753───X^0.5───────────────────────────────────────────────────────────────@────────────(Z-Y^-0.5)───@────────────[Z]^-0.498───X^0.5───@─────────────[Z]^-0.497───X^0.5────────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@────────(Z-Y^-0.5)─────────────────────────────────────────────────────────────@─────────────[Z]^0.00196─────[X]^0.999───@───────────[X]^0.999────────────@───────────[Z]^-0.501───X^0.5────────────────────────────────────────────────@───────────(Z-Y^-0.5)───@─────────────[Z]^-0.498───X^0.5───@────────────[Z]^-0.00671───X^0.5──────────────────────────@───────[X]^-0.502───@────────────[X]^0.00622─────────────────@────────(Z-Y^-0.5)──────────────────────────────────────────────────────────────────────────@────────────[Z]^-0.00311───[X]^0.999───@────────────[X]^0.999─────────────@─────────────[Z]^0.998───X^0.5───────────────────────────────────────────────────────────────@───────────(Z-Y^-0.5)───@────────────[Z]^-0.498───X^0.5───@──────────────[Z]^-0.497───X^0.5────────────────────────────@───────[X]^-0.502───@────────────[X]^-0.00392──────────────────@───────(Z-Y^-0.5)───────────────────────────────────────────────────────────@───────────[Z]^0.252───────[X]^0.997───@────────────[X]^-0.997───────────@───────────[Z]^0.753───X^0.5──────────────────────────────────────────────────────

In [12]:
# Define a phase estimation circuit.
def measure_bit_of_phase(system_qubits,
                         control_qubit,
                         controlled_unitary):
    yield cirq.H(control_qubit)
    yield controlled_unitary
    yield cirq.H(control_qubit)
    yield cirq.measure(control_qubit)

# Get an upper bound on the Hamiltonian norm.
import numpy
bound = numpy.sum(numpy.abs(hamiltonian.one_body)) + numpy.sum(numpy.abs(hamiltonian.two_body))

# Construct phase estimation circuit.
time = 2 * numpy.pi / bound
control = cirq.LineQubit(-1)

controlled_unitary = openfermioncirq.simulate_trotter(
    qubits, hamiltonian, time,
    n_steps=1,
    order=1,
    algorithm=trotter.LINEAR_SWAP_NETWORK,
    control_qubit=control)

circuit = cirq.Circuit.from_ops(
    measure_bit_of_phase(
        qubits,
        control,
        controlled_unitary))

# Print the circuit.
cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

-1 0         1            2         3
│  │         │            │         │
H  │         │            │         │
│  │         │            │         │
@──XXYY──────XXYY^-0.0484 │         │
│  │         │            │         │
@──@─────────@^0.0613     │         │
│  │         │            │         │
│  ×ᶠ────────×ᶠ           │         │
│  │         │            │         │
@──┼─────────┼────────────XXYY──────XXYY^-0.0484
│  │         │            │         │
@──┼─────────┼────────────@─────────@^0.0613
│  │         │            │         │
│  │         │            ×ᶠ────────×ᶠ
│  │         │            │         │
@──┼─────────@────────────@^0.184   │
│  │         │            │         │
│  │         ×ᶠ───────────×ᶠ        │
│  │         │            │         │
@──XXYY──────XXYY^-0.0484 │         │
│  │         │            │         │
@──@─────────@^0.0613     │         │
│  │         │            │         │
│  ×ᶠ────────×ᶠ           │         │
│  │         │            │    

In [13]:
c_opt = optimize(circuit)

Before
Failed to optimize for Xmon
  CZ count: 200
  Depth: 391
After
  CZ count: 128
  Depth: 200


-1: ───H───@──────────────@───────────────@──────────────@───────────────@──────────────@──────────────@───────────────@──────────────@───────────────@──────────────@───────────@───────────@───────────@───────────@──────────────@──────────@──────────────@──────────@───────────────────@──────────────@──────────@──────────────@──────────@───────────────────H───M───
           │              │               │              │               │              │              │               │              │               │              │           │           │           │           │              │          │              │          │                   │              │          │              │          │
0: ────────XXYY───────────@──────────×ᶠ───┼──────────────┼───────────────┼──────────────XXYY───────────@──────────×ᶠ───┼──────────────┼───────────────┼──────────────┼───────────┼───────────┼───────────@^-0.0968───┼──────────────┼──────────┼──────────────@──────────XXYY───────────×ᶠ───┼──────────────┼──────────┼──────────────@──────────XXYY───────────×ᶠ───────────
           │              │          │    │              │               │              │              │          │    │              │               │              │           │           │                       │              │          │              │          │              │    │              │          │              │          │              │
1: ────────XXYY^-0.0484───@^0.0613───×ᶠ───┼──────────────┼───────────────@─────────×ᶠ───XXYY^-0.0484───@^0.0613───×ᶠ───┼──────────────┼───────────────@─────────×ᶠ───┼───────────┼───────────@^-0.0968───────────────@─────────×ᶠ───┼──────────┼──────────────@^0.0613───XXYY^-0.0484───×ᶠ───@─────────×ᶠ───┼──────────┼──────────────@^0.0613───XXYY^-0.0484───×ᶠ───────────
                                          │              │               │         │                                   │              │               │         │    │           │                                   │         │    │          │                                             │         │    │          │
2: ───────────────────────────────────────XXYY───────────@──────────×ᶠ───@^0.184───×ᶠ──────────────────────────────────XXYY───────────@──────────×ᶠ───@^0.184───×ᶠ───┼───────────@^-0.0968───────────────────────────@^0.184───×ᶠ───@──────────XXYY───────────×ᶠ─────────────────────────────@^0.184───×ᶠ───@──────────XXYY───────────×ᶠ─────────────────────────────────────
                                          │              │          │                                                  │              │          │                   │                                                              │          │              │                                             │          │              │
3: ───────────────────────────────────────XXYY^-0.0484───@^0.0613───×ᶠ─────────────────────────────────────────────────XXYY^-0.0484───@^0.0613───×ᶠ──────────────────@^-0.0968──────────────────────────────────────────────────────@^0.0613───XXYY^-0.0484───×ᶠ────────────────────────────────────────────@^0.0613───XXYY^-0.0484───×ᶠ─────────────────────────────────────

-1: ───Y^0.5───@────────────────────@─────────────────────────@────────────────────────@─────────────────────────────────────────────────@────────────────────@───────────@────────────────────────────────@───────────────────────────────────────@────────────────────────@─────────────────────────────────────────────────@──────────────────────────────────@────────────────────────────────────────────────────────@────────────────────@───X^0.5───@───[Y]^0.0613───@───X^-0.5───@────────────────────────────────────────────────────@────────────────────────@─────────────────────────@───────@────────────────────────────────────────────────────@─────────────────────────────────────────@────────────────────────@─────────────────────────────────────────@────────────────@─────────────────────────────────@─────────────────@───X^-0.5───@───[Y]^0.0742───@───X^0.5─────────────────────────@────────────────────────────────────@─────────────────────────────────────────────────────────@─────────────────────@───@─────────────────────────@───Y^-0.5───@───[X]^0.0177───@───(Z-Y^-0.5)─────────────────────────@─────────────────────────────────────@──────────────────────────────────────────────────@─────────────────────@──────────────────────────────────────────────────────@───────────────────@──────────────────────────────────────────────────@─────────────────@───Y^-0.5───@───[X]^-0.0613───@───(Z-Y^-0.5)────────────────────────@─────────────────────────────────────@─────────────────────────────────────────────────@─────────────────────@───@─────────────────────────@───────X^-0.5───────@───[Y]^0.0153───@───X^0.5────────────────────────@───────────────────────────────────────@─────────────────────────────────────────@────────────────────────────@───[Z]^-0.613───(X^0.5-Z^-0.5)───M───────────────
               │                    │                         │                        │                                                 │                    │           │                                │                                       │                        │                                                 │                                  │                                                        │                    │           │                │            │                                                    │                        │                         │       │                                                    │                                         │                        │                                         │                │                                 │                 │            │                │                                 │                                    │                                                         │                     │   │                         │            │                │                                      │                                     │                                                  │                     │                                                      │                   │                                                  │                 │            │                 │                                     │                                     │                                                 │                     │   │                         │                    │                │                                │                                       │                                         │                            │
0: ────────────@───@───[X]^0.0121───@───@───(Z^-0.5-X^-0.5)───@───@───[X]^0.0121───@───@───────────────(Z^0.5-X^0.5)───@───[X]^0.0153────@───[X]^0.0153───@───@───────@───┼───(Z^0.5-X^0.5)───@────────────┼───[Y]^-0.0153─────────────────────────┼────────────────────────┼─────────────────────────────────────────────────┼──────────────────────────────────┼────────────────────────────────────────────────────────┼────────────────────┼───────────┼────────────────┼──